In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/DCASE-task1"

/content/drive/MyDrive/DCASE-task1


In [ ]:
# from zipfile import ZipFile
# with ZipFile("audio.zip", 'r') as zip:
#     # printing all the contents of the zip file
  
#     # extracting all the files
#     print('Extracting all the files now...')
#     zip.extractall()
#     print('Done!')

In [ ]:
sampling_rate = 44100
row_len = 513 # Number of columns: 1 + n_fft/2
col_len = 401 # Number of rows: 1 + (sampling_rate*audio_duration)/(0.01*sampling_rate); 0.01*sampling_rate = hop 
# audio_length = sampling_rate*audio_duration
# audio_duration = 4 seconds
folder = "train"

In [ ]:
import numpy as np
import pandas as pd

from os import path
import librosa
from keras.utils.np_utils import to_categorical

import tensorflow as tf

In [ ]:
def wav2feat(wavfile, Fs = sampling_rate):
    x, _ = librosa.core.load(wavfile, sr = Fs, mono = True)
    hop = int(0.01*Fs) # 10ms
    win = int(0.02*Fs) # 20ms
    X = librosa.stft(x, n_fft = 1024, hop_length = hop, win_length = win, window = 'hann', center = True, pad_mode = "reflect")
    X = np.abs(X)

    if X.shape[0] > row_len:
        max_offset = X.shape[0] - row_len
        offset = np.random.randint(max_offset)
        X = X[offset : (row_len + offset), :]
    else:
        if X.shape[0] < row_len:
            max_offset = row_len - X.shape[0]
            offset = np.random.randint(max_offset)
        else:
            offset = 0
        X = np.pad(X, ((offset, row_len - X.shape[0] - offset), (0, 0)), "constant")

    if X.shape[1] > col_len:
        max_offset = X.shape[1] - col_len
        offset = np.random.randint(max_offset)
        X = X[:, offset : (col_len + offset)]
    else:
        if X.shape[1] < col_len:
            max_offset = col_len - X.shape[1]
            offset = np.random.randint(max_offset)
        else:
            offset = 0
        X = np.pad(X, ((0, 0), (offset, col_len - X.shape[1] - offset)), "constant")

    return X

In [ ]:
def prepare_data():
    print("Number of training samples processed: ")
    Y = to_categorical(train_df["label_idx"], num_classes = n_classes)
    for i, fname in (train_df[["file_name"]].iterrows()):
        fpath = fname[0]
        
        try:
            path.exists(fpath)
            STFT = wav2feat(fpath)
            STFT = np.expand_dims(STFT, axis = -1)

            np.save(folder + "/STFT%d.npy"%i, STFT)
            np.save(folder + "/target%d.npy"%i, Y[i])
        except:
            print("Does not exist:", fpath, "index:", i)

        if(i != 0 and i%200 == 0):
            print(i, end = ".. ")

    print(train_df.shape[0], end = ".. ")
    print("Done!")

    return

In [ ]:
train_df = pd.read_csv("dcase1_" + folder + ".csv")
train_df.head(5)

,file_name,class
0,audio/shopping_mall-barcelona-254-7636-a.wav,shopping_mall
1,audio/shopping_mall-helsinki-255-7652-b.wav,shopping_mall
2,audio/shopping_mall-barcelona-125-3702-a.wav,shopping_mall
3,audio/shopping_mall-lyon-1043-40055-a.wav,shopping_mall
4,audio/shopping_mall-barcelona-126-3748-s2.wav,shopping_mall


In [ ]:
train_df = pd.read_csv("dcase1_train.csv")
LABELS = list(train_df["class"].unique())
label_idx = {label: i for i, label in enumerate(LABELS)}
n_classes = len(train_df["class"].unique())

train_df = pd.read_csv("dcase1_" + folder + ".csv")
train_df["label_idx"] = train_df["class"].apply(lambda x : label_idx[x])

In [ ]:
print("Number of .wav files = ", train_df.shape[0])

Number of .wav files =  7150


In [ ]:
label_idx

{'airport': 7,
 'bus': 5,
 'metro': 2,
 'metro_station': 4,
 'park': 3,
 'public_square': 8,
 'shopping_mall': 0,
 'street_pedestrian': 1,
 'street_traffic': 6,
 'tram': 9}

In [ ]:
prepare_data()

Number of training samples processed: 
200.. 400.. 600.. 800.. 1000.. 1200.. 1400.. 1600.. 1800.. 2000.. 2200.. 2400.. 2600.. 2800.. 3000.. 3200.. 3400.. 3600.. 3800.. 4000.. 4200.. 4400.. 4600.. 4800.. 5000.. 5200.. 5400.. 5600.. 5800.. 6000.. 6200.. 6400.. 6600.. 6800.. 7000.. 7150.. Done!


In [ ]:
stft_shape = np.load("train/STFT0.npy").shape
target_shape = np.load("train/target0.npy").shape
print(stft_shape, target_shape)

(513, 401, 1) (10,)


In [ ]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list = tf.train.BytesList(value = [value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def serialize_example(feature0, feature1):
    feature = {
      "STFT": _bytes_feature(feature0),
      "target": _bytes_feature(feature1)
      }
    
    example_proto = tf.train.Example(features=tf.train.Features(feature = feature))
    
    return example_proto.SerializeToString()

In [ ]:
LEN = train_df.shape[0]

In [ ]:
SIZE = 250
CT = LEN//SIZE + int(LEN%SIZE != 0)

for j in range(CT):
    print(); print('Writing TFRecord %i of %i...'%(j + 1, CT))
    CT2 = min(SIZE , LEN - j*SIZE)
    
    with tf.io.TFRecordWriter(folder + "fold1_%.2i-%i.tfrec"%(j, CT2)) as writer:
        for k in range(CT2):
            # img = cv2.imread(PATH + IMGS_train1[SIZE*j + k] + '.jpg')
            # img = cv2.imencode('.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, 94))[1].tostring()
            idx = SIZE*j + k
            X_train = np.load(folder + "/STFT%d.npy"%idx)
            Y_train = np.load(folder + "/target%d.npy"%idx)

            assert X_train.shape == stft_shape
            assert Y_train.shape == target_shape

            X_train = X_train.tobytes()
            Y_train = Y_train.tobytes()

            example = serialize_example(X_train, Y_train)           
            
            writer.write(example)
            if k%100 == 0: print(k, ', ', end = '')


Writing TFRecord 1 of 29...
0 , 100 , 200 , 
Writing TFRecord 2 of 29...
0 , 100 , 200 , 
Writing TFRecord 3 of 29...
0 , 100 , 200 , 
Writing TFRecord 4 of 29...
0 , 100 , 200 , 
Writing TFRecord 5 of 29...
0 , 100 , 200 , 
Writing TFRecord 6 of 29...
0 , 100 , 200 , 
Writing TFRecord 7 of 29...
0 , 100 , 200 , 
Writing TFRecord 8 of 29...
0 , 100 , 200 , 
Writing TFRecord 9 of 29...
0 , 100 , 200 , 
Writing TFRecord 10 of 29...
0 , 100 , 200 , 
Writing TFRecord 11 of 29...
0 , 100 , 200 , 
Writing TFRecord 12 of 29...
0 , 100 , 200 , 
Writing TFRecord 13 of 29...
0 , 100 , 200 , 
Writing TFRecord 14 of 29...
0 , 100 , 200 , 
Writing TFRecord 15 of 29...
0 , 100 , 200 , 
Writing TFRecord 16 of 29...
0 , 100 , 200 , 
Writing TFRecord 17 of 29...
0 , 100 , 200 , 
Writing TFRecord 18 of 29...
0 , 100 , 200 , 
Writing TFRecord 19 of 29...
0 , 100 , 200 , 
Writing TFRecord 20 of 29...
0 , 100 , 200 , 
Writing TFRecord 21 of 29...
0 , 100 , 200 , 
Writing TFRecord 22 of 29...
0 , 100 , 200